In [10]:
from abc import ABC,abstractmethod
import pandas as pd
import numpy as np

In [11]:
class ColumnNotExistException(Exception):
    pass

In [12]:
class Validator():
    '''wrapper class to run multiple validators'''
    def __init__(self,validators=[]):
        self.validators = validators
        

In [13]:
class BaseColumnValidator(ABC):
    '''Base class for column validator'''
    #all indices that did not pass the validation
    failed_indices = []

    #this is the dataframe that's being passed to the next validator
    validated_df = None

    @abstractmethod  
    def get_failed_indices(self,df,field):
        '''this function should return all failed indices'''
        pass
    
    def validate(self,df,field):
        self.field = field
        self.validated_df = self.add_errors(df,self.get_failed_indices(df,field))
        
        
    def is_column_exist(self,df,field):
        '''checks if column exist on database'''
        if field in df.columns:
            return True

        #return the original dataframe
        self.validated_df = df
        return False
        
   
        
    def add_errors(self,df, failed_indices):
        '''this adds error column on validated dataframe'''
        if failed_indices == []:
            return df
        with_error = df

        if 'error' not in df.columns:
            with_error['error'] = ""
            
        with_error.loc[failed_indices, 'error'] = with_error.loc[failed_indices, 'error'] + f", {self.field} - {self.error_message}" 
        return with_error
            

In [14]:
class IntegerValidator(BaseColumnValidator):
    error_message = "Not an integer"

    def get_failed_indices(self,df,field):
        if self.is_column_exist(df,field):
            df[field] = pd.to_numeric(df[field],errors="coerce")
            filtered_df = df[df[field].isna()]
            return list(filtered_df.index.values)
        

In [15]:
class StringValidator(BaseColumnValidator):
    '''this does not validate, just converting column value as string'''
    def __init__(self,max_length=None):
        self.max_length = max_length
        self.error_message = f"has more than {max_length} characters"
    def get_failed_indices(self,df,field):
        if self.is_column_exist(df,field):
            ##convert all data to strings
            df[field] = df[field].fillna("")
            df[field] = df[field].astype(str)
            filtered = df[df[field].str.len() > self.max_length]
            
            return list(filtered.index.values)
            

In [16]:
class RequiredValidator(BaseColumnValidator):
    error_message = "is required"
    def get_failed_indices(self,df,field):
        if self.is_column_exist(df,field):
            nan_values = list(df.loc[df[field].isna()].index.values)
            empty_string =  list(df.loc[df[field] == ""].index.values)
            failed_indices = nan_values + empty_string
            return list(set(failed_indices))

In [17]:
df = pd.read_csv("csv_upload.csv")

In [106]:
class BaseDataFrameValidator(ABC):
    ''''this is a dataframe level validator used to validate asset dataframe'''
    def __init__(self,df,**kwargs) -> None:
        self.df = df
        self.validated_df = None
        self.failed_indices = []
        self.required_fields = []

    def append_to_failed_indices(self,indices):
        self.failed_indices += indices
        self.failed_indices = list(set(self.failed_indices))

    def run_column_validators(self):
        '''this handles running of multiple validators'''
        fields = [(x,getattr(self, x)) for x in dir(self)]

        #get all validator fields
        column_validators = [x for x in fields
                             if isinstance(x[1], Validator)]
        
        to_filter_df = self.df
        for field_name , validator in column_validators:
            column_validators = validator.validators
            
            for column_validator in column_validators:
                column_validator.validate(to_filter_df,field_name)
                to_filter_df = column_validator.validated_df
                
                self.append_to_failed_indices(column_validator.failed_indices)
            self.validated_df = column_validator.validated_df
        
    def validate(self):
        '''this is the entry point method to all class'''
        self.run_column_validators()
        

    def get_required_fields(self):
        return self.required_fields

In [107]:
class AssetDataframeValidator(BaseDataFrameValidator):
    Nan1 = Validator(validators = [RequiredValidator()])
    Nan2 = Validator(validators = [IntegerValidator()])
    Serial = Validator(validators = [StringValidator(max_length = 5),IntegerValidator()])

In [108]:
x = AssetDataframeValidator(df = df)
x.validate()

In [109]:
x.validated_df

,Name,Serial,date,Nan1,Nan2,error
0,Test_0,1100231,11/11/2024,t,NaN,", Nan2 - Not an integer, Serial - has more tha..."
1,Test_2,11234,12/11/2024,NaN,2.0,", Nan1 - is required"
2,Test_2,11234,12/11/2024,11,2.0,
3,Test_2,11234,12/11/2024,NaN,NaN,", Nan1 - is required, Nan2 - Not an integer"


In [90]:
x.validated_df.to_json(orient='records')

'[{"Name":"Test_0","Serial":1100231,"date":"11\\/11\\/2024","Nan1":"t","Nan2":null,"error":", Nan2 - Not an integer, Nan2 - Not an integer, Nan2 - Not an integer, Nan2 - Not an integer"},{"Name":"Test_2","Serial":11234,"date":"12\\/11\\/2024","Nan1":"","Nan2":2.0,"error":", Nan1 - is required, Nan1 - is required, Nan1 - is required, Nan1 - is required"},{"Name":"Test_2","Serial":11234,"date":"12\\/11\\/2024","Nan1":"11","Nan2":2.0,"error":""},{"Name":"Test_2","Serial":11234,"date":"12\\/11\\/2024","Nan1":"","Nan2":null,"error":", Nan1 - is required, Nan2 - Not an integer, Nan1 - is required, Nan2 - Not an integer, Nan1 - is required, Nan2 - Not an integer, Nan1 - is required, Nan2 - Not an integer"}]'

In [91]:
df

,Name,Serial,date,Nan1,Nan2,error
0,Test_0,1100231,11/11/2024,t,NaN,", Nan2 - Not an integer, Nan2 - Not an integer..."
1,Test_2,11234,12/11/2024,,2.0,", Nan1 - is required, Nan1 - is required, Nan1..."
2,Test_2,11234,12/11/2024,11,2.0,
3,Test_2,11234,12/11/2024,,NaN,", Nan1 - is required, Nan2 - Not an integer, N..."


In [96]:
df[df["Name"].str.len() == 4]

,Name,Serial,date,Nan1,Nan2,error


In [110]:
df.dtypes()

TypeError: 'Series' object is not callable

In [111]:
df

,Name,Serial,date,Nan1,Nan2,error
0,Test_0,1100231,11/11/2024,t,NaN,", Nan2 - Not an integer, Serial - has more tha..."
1,Test_2,11234,12/11/2024,NaN,2.0,", Nan1 - is required"
2,Test_2,11234,12/11/2024,11,2.0,
3,Test_2,11234,12/11/2024,NaN,NaN,", Nan1 - is required, Nan2 - Not an integer"


In [112]:
df.dtypes

Name       object
Serial      int64
date       object
Nan1       object
Nan2      float64
error      object
dtype: object

In [96]:
df = pd.read_csv("csv_upload.csv")

In [97]:
df

,Name,Serial,date,Nan1,Nan2
0,Test_0,1100231,11/11/2024,t,NaN
1,Test_2,11234,12/11/2024,NaN,2
2,Test_2,11234,12/11/2024,11,0
3,Test_2,11234,12/11/2024,NaN,aafafa


In [98]:
df['date'] = pd.to_datetime(df['date'], format='%m/%d/%Y', errors='coerce')

In [99]:
df[df["date"].isna()]

,Name,Serial,date,Nan1,Nan2


In [8]:
class test():
    attrb = "asd"
    def __init__(self):
        lookup = {
        "attrb":setattr(self,"test","testv")
        }
        lookup.get("attrb")

In [9]:
test().test

'testv'

In [51]:
"attrb" in dir(test)

True

In [43]:
asset_fields = {
    "fileupload": {
        "type": "ForeignKey",
        "max_length": ""
    },
    "revnueassetfiles": {
        "type": "ForeignKey",
        "max_length": ""
    },
    "children_assets": {
        "type": "ManyToManyField",
        "max_length": ""
    },
    "revnueassethistory": {
        "type": "ForeignKey",
        "max_length": ""
    },
    "requestprocurementasset": {
        "type": "ForeignKey",
        "max_length": ""
    },
    "customformasset": {
        "type": "ForeignKey",
        "max_length": ""
    },
    "customfieldasset": {
        "type": "ForeignKey",
        "max_length": ""
    },
    "purchaseorderasset": {
        "type": "ForeignKey",
        "max_length": ""
    },
    "assetlocation": {
        "type": "ForeignKey",
        "max_length": ""
    },
    "assetbusinessentity": {
        "type": "ForeignKey",
        "max_length": ""
    },
    "created_by": {
        "type": "ForeignKey",
        "max_length": None
    },
    "modified_date": {
        "type": "DateTimeField",
        "max_length": None
    },
    "modified_by": {
        "type": "ForeignKey",
        "max_length": None
    },
    "created_date": {
        "type": "DateTimeField",
        "max_length": None
    },
    "company": {
        "type": "ForeignKey",
        "max_length": None
    },
    "contract_name": {
        "type": "CharField",
        "max_length": 50
    },
    "contract_number": {
        "type": "CharField",
        "max_length": 50
    },
    "supplier_name": {
        "type": "CharField",
        "max_length": 50
    },
    "customer_name": {
        "type": "CharField",
        "max_length": 50
    },
    "contract": {
        "type": "ForeignKey",
        "max_length": None
    },
    "asset_id": {
        "type": "AutoField",
        "max_length": None
    },
    "asset_name": {
        "type": "CharField",
        "max_length": 200
    },
    "category": {
        "type": "ForeignKey",
        "max_length": None
    },
    "subcategory": {
        "type": "CharField",
        "max_length": 50
    },
    "asset_hardware_model": {
        "type": "ForeignKey",
        "max_length": None
    },
    "supplier": {
        "type": "ForeignKey",
        "max_length": None
    },
    "customer": {
        "type": "ForeignKey",
        "max_length": None
    },
    "customer_part_number": {
        "type": "CharField",
        "max_length": 50
    },
    "manufacturer": {
        "type": "CharField",
        "max_length": 50
    },
    "serial_number": {
        "type": "CharField",
        "max_length": 50
    },
    "parent_manufacturer_part_number": {
        "type": "CharField",
        "max_length": 50
    },
    "parent_serial_number": {
        "type": "CharField",
        "max_length": 50
    },
    "manufacturer_part_number": {
        "type": "CharField",
        "max_length": 50
    },
    "hardware_model": {
        "type": "CharField",
        "max_length": 50
    },
    "sales_manager_name": {
        "type": "CharField",
        "max_length": 50
    },
    "sales_manager_email": {
        "type": "CharField",
        "max_length": 254
    },
    "additional_sales_email": {
        "type": "CharField",
        "max_length": 254
    },
    "contract_service_per_item": {
        "type": "DecimalField",
        "max_length": None
    },
    "general_info_notes": {
        "type": "CharField",
        "max_length": 1000
    },
    "business_entity": {
        "type": "ForeignKey",
        "max_length": None
    },
    "supplier_manufacturer": {
        "type": "ForeignKey",
        "max_length": None
    },
    "service_provider": {
        "type": "ForeignKey",
        "max_length": None
    },
    "supplier_distributor": {
        "type": "ForeignKey",
        "max_length": None
    },
    "supplier_reseller": {
        "type": "ForeignKey",
        "max_length": None
    },
    "service_type": {
        "type": "ForeignKey",
        "max_length": None
    },
    "business_entities": {
        "type": "JSONField",
        "max_length": None
    },
    "business_entity_types": {
        "type": "CharField",
        "max_length": 1000
    },
    "address": {
        "type": "CharField",
        "max_length": 1000
    },
    "city": {
        "type": "CharField",
        "max_length": 50
    },
    "state_province": {
        "type": "CharField",
        "max_length": 50
    },
    "country": {
        "type": "ForeignKey",
        "max_length": None
    },
    "region": {
        "type": "CharField",
        "max_length": 50
    },
    "zip_code": {
        "type": "CharField",
        "max_length": 20
    },
    "asset_location_address": {
        "type": "CharField",
        "max_length": 200
    },
    "location_description": {
        "type": "CharField",
        "max_length": 1000
    },
    "latitude": {
        "type": "DecimalField",
        "max_length": None
    },
    "longitude": {
        "type": "DecimalField",
        "max_length": None
    },
    "street": {
        "type": "CharField",
        "max_length": 50
    },
    "bldg_name": {
        "type": "CharField",
        "max_length": 50
    },
    "floor": {
        "type": "CharField",
        "max_length": 50
    },
    "unit": {
        "type": "CharField",
        "max_length": 50
    },
    "room": {
        "type": "CharField",
        "max_length": 50
    },
    "area": {
        "type": "CharField",
        "max_length": 50
    },
    "asset_owner": {
        "type": "ForeignKey",
        "max_length": None
    },
    "business_unit": {
        "type": "ForeignKey",
        "max_length": None
    },
    "asset_owner_email": {
        "type": "CharField",
        "max_length": 254
    },
    "asset_owner_phone": {
        "type": "CharField",
        "max_length": 20
    },
    "warranty_type": {
        "type": "CharField",
        "max_length": 50
    },
    "warranty_provider": {
        "type": "ForeignKey",
        "max_length": None
    },
    "warranty_eol_date": {
        "type": "DateTimeField",
        "max_length": None
    },
    "warranty_start_date": {
        "type": "DateTimeField",
        "max_length": None
    },
    "warranty_end_date": {
        "type": "DateTimeField",
        "max_length": None
    },
    "warranty_contact_phone": {
        "type": "CharField",
        "max_length": 20
    },
    "warranty_contact_email": {
        "type": "CharField",
        "max_length": 254
    },
    "warranty_contact_notes": {
        "type": "CharField",
        "max_length": 1000
    },
    "support_type": {
        "type": "CharField",
        "max_length": 50
    },
    "support_provider": {
        "type": "ForeignKey",
        "max_length": None
    },
    "support_eol_date": {
        "type": "DateTimeField",
        "max_length": None
    },
    "support_start_date": {
        "type": "DateTimeField",
        "max_length": None
    },
    "support_end_date": {
        "type": "DateTimeField",
        "max_length": None
    },
    "support_contact_phone": {
        "type": "CharField",
        "max_length": 20
    },
    "support_contact_email": {
        "type": "CharField",
        "max_length": 254
    },
    "support_contact_notes": {
        "type": "CharField",
        "max_length": 1000
    },
    "usage_status": {
        "type": "CharField",
        "max_length": 50
    },
    "asset_tag": {
        "type": "CharField",
        "max_length": 50
    },
    "asset_notes": {
        "type": "CharField",
        "max_length": 1000
    },
    "currency": {
        "type": "JSONField",
        "max_length": None
    },
    "original_cost": {
        "type": "DecimalField",
        "max_length": None
    },
    "new_hardware_selling_unit_price": {
        "type": "DecimalField",
        "max_length": None
    },
    "service_start_date": {
        "type": "DateTimeField",
        "max_length": None
    },
    "estimated_salvage_value": {
        "type": "DecimalField",
        "max_length": None
    },
    "depreciation_schedule": {
        "type": "CharField",
        "max_length": 50
    },
    "financial_notes": {
        "type": "CharField",
        "max_length": 1000
    },
    "manufacturer_currency": {
        "type": "JSONField",
        "max_length": None
    },
    "manufacturer_warranty_unit_price": {
        "type": "DecimalField",
        "max_length": None
    },
    "template": {
        "type": "ForeignKey",
        "max_length": None
    },
    "is_external": {
        "type": "BooleanField",
        "max_length": None
    },
    "currency_code": {
        "type": "CharField",
        "max_length": 1000
    },
    "conversion_rate_usd": {
        "type": "DecimalField",
        "max_length": None
    },
    "sold_under_contract": {
        "type": "ForeignKey",
        "max_length": None
    },
    "attached_supplier": {
        "type": "ForeignKey",
        "max_length": None
    },
    "asset_usage_status": {
        "type": "ForeignKey",
        "max_length": None
    },
    "asset_obj_type": {
        "type": "CharField",
        "max_length": 50
    },
    "asset_image": {
        "type": "CharField",
        "max_length": 255
    },
    "building_name": {
        "type": "CharField",
        "max_length": 255
    },
    "upload_queue": {
        "type": "ForeignKey",
        "max_length": None
    },
    "alert": {
        "type": "ManyToManyField",
        "max_length": None
    },
    "parent_assets": {
        "type": "ManyToManyField",
        "max_length": None
    }
}

In [48]:
list(set([asset_fields[key]["type"] for key in asset_fields]))

['AutoField',
 'DecimalField',
 'CharField',
 'DateTimeField',
 'ManyToManyField',
 'ForeignKey',
 'BooleanField',
 'JSONField']

In [30]:
df

,Name,Serial,date,Nan1,Nan2,asd
0,Test_0,1100231,11/11/2024,t,NaN,test
1,Test_2,11234,12/11/2024,NaN,2,test
2,Test_2,11234,12/11/2024,11,2,test
3,Test_2,11234,12/11/2024,NaN,aafafa,test


In [25]:
g

,Name,Serial,date,Nan1,Nan2
0,Test_0,1100231,11/11/2024,t,NaN
2,Test_2,11234,12/11/2024,11,2


In [ ]:
df